# Project Overview
In this notebook and subsequent notebooks I will attempt to predict mechanically meaningful labels associated with some fan-generated roleplaying game (RPG) content.

I collected post and comment data from Reddit in multiple runs using a command-line tool I wrote for this purpose (see: [PRAW-CoDiaLS](https://github.com/nkuehnle/praw-codials))

The actual text I will be attempting to predict is hosted on one of two websites: [GM Binder](https://gmbinder.com/) or [Homebrewery](https://homebrewery.naturalcrit.com/), which are tools/content hosting providers for fan-made RPG content, primarily Dungeons and Dragons 5th Edition (DnD5e).

The labels make use of the user-assigned "flair" from the subreddit [/r/UnearthedArcana](https://www.reddit.com/r/UnearthedArcana/). UnearthedArcana describes itself as a source of "homebrew" (fan-generated) content and its community rules ensure that every post has a meaningful label ("flair") associated with it. These labels describe the "type" of game content that has been created, be it a character "class" (think achetypal high-fantasy characters: knights, wizards, archers, etc) or a "race" (elves, dwarves, humans, etc).

At the start of this document we have only the data about the Reddit submission including the submission flair link to the service described above.

In [1]:
%load_ext jupyter_black
# Utility Imports
import os
from pathlib import Path

# Imports for data processing/handling
import pandas as pd
from urllib.parse import urlparse
import itertools

# This Notebook
This notebook will clean up the individual links and their associated labels that were collected from Reddit.

Below, I describe some over-arching issues with the links collected within the dataset and start to fix them.

In [2]:
# Define a constant (CWD) to use
CWD = Path(os.getcwd())
DATA = CWD / "data"
PRAW = DATA / "praw_data"

for f in [DATA, PRAW]:
    fname = str(f)
    currname = str(CWD)
    fname = fname.replace(currname, "")
    try:
        f.mkdir(parents=True, exist_ok=False)
    except FileExistsError:
        pass
    else:
        print(f"Folder {fname} was created")

# Data Sources
There are two files (CommentsRaw.csv and SubmissionsRaw.csv)

If you are not familiar with Reddit, when users post it can take two forms, "Submissions" and "comments."

Submissions are associated with a specific community or "subreddit" and users can comment on Submissions to discuss them or provide additional commentary.

Submissions can come in multiple forms: link, text, image, etc. The submissions here are only link submissions. Comments are searched for all other types of submissions as users will frequently post an image version of their 
content and then link to the text copy in the comments on these subreddits. The vast majority of content on /r/UnearthedArcana comes in this second form (an image submission with the link in the comments).

In any case, PRAW-CoDiaLS is written so that they both share the same overall structure, although certain fields are only used for comments (and blank otherwise).


### Shared Fields

| column name | description |
| ----- | ----- |
| link | Raw text URL |
| submission_author | Reddit user who posted the comment |
| submission_id | A unique ID used for locating the submission on Reddit |
| submission_title | The title of the submission on Reddit |
| subreddit | The name of the Subreddit the content was found on (in this case, either UnearthedArcana or DnDHomebrew) |
| submission_flair | the figure number as indicated in the manuscript |
| submission_score | The approximate number of net positive votes received by the submission on Reddit, the actual values are obfuscated by Reddit to avoid manipulation |
| submission_upvote_ratio | The ratio of positive to negative votes |
| submission_date | Date on which the submission was originally made |

### Comment-only fields

| column name | description |
| ----- | ----- |
| comment_author | Reddit user who posted the comment |
| comment_id | A unique ID used for locating the comment on Reddit |
| comment_score | The approximate number of net positive votes received by the comment on Reddit, the actual values are obfuscated by Reddit to avoid manipulation |
| comment_body | Text of the comment containing markdown elements |
| comment_date | Date on which the comment was originally posted |

### Other Notes
There are many cases where matching a URL and flair are not entirely straightforward. I will discuss this as it comes up below.

In [3]:
subs_path = PRAW / "SubmissionsRaw.csv"
subs_df = pd.read_csv(subs_path, sep=",")  # Load in submission data
subs_df["submission_flair"] = subs_df["submission_flair"].replace("Feature", "Feat")
subs_df = subs_df.drop_duplicates(subset=["link", "submission_id"])
subs_df = subs_df.reset_index(drop=True)
subs_df.to_csv(subs_path, index=False)  # Save data
subs_df.head()

,link,submission_author,submission_id,submission_title,subreddit,submission_flair,submission_score,submission_upvote_ratio,submission_date,comment_author,comment_id,comment_score,comment_body,comment_date
0,https://www.gmbinder.com/share/-Me6GgAeQEHyXMS...,Barnopottamous,ok0zzk,The Mystic Knight - A Martial Arcane Class for...,UnearthedArcana,Class,22.0,0.92,1.626256e+09,NaN,NaN,NaN,NaN,NaN
1,http://homebrewery.naturalcrit.com/share/SJbQc...,TheIncredibleMrT,8jgdjk,Need some feedback on the Dhampir race me and ...,UnearthedArcana,Race,0.0,0.50,1.526336e+09,NaN,NaN,NaN,NaN,NaN
2,https://www.gmbinder.com/share/-L6WsN8LHe6K8uk...,Zenrayeed,8nfqar,College of Hope v1.2: A Healing-Focused Bard C...,UnearthedArcana,Subclass,0.0,0.50,1.527744e+09,NaN,NaN,NaN,NaN,NaN
3,http://homebrewery.naturalcrit.com/share/SyF38...,NaN,8qofe5,Nature's Order: A Ranger Conclave,UnearthedArcana,Subclass,0.0,0.43,1.528853e+09,NaN,NaN,NaN,NaN,NaN
4,https://www.gmbinder.com/share/-LG5mpzIVA2Ubjh...,hugglon,8uke6l,Circle of Voodoo [Feedback Appreciated],UnearthedArcana,Subclass,0.0,0.36,1.530199e+09,NaN,NaN,NaN,NaN,NaN


In [4]:
cmts_path = PRAW / "CommentsRaw.csv"
cmts_df = pd.read_csv(cmts_path, sep=",")  # Load comment data
cmts_df["submission_flair"] = cmts_df["submission_flair"].replace("Feature", "Feat")
cmts_df = cmts_df.drop_duplicates(subset=["link", "comment_id"])
cmts_df = cmts_df.reset_index(drop=True)
cmts_df.to_csv(cmts_path, index=False)  # Save data
cmts_df.head()

,link,submission_author,submission_id,submission_title,subreddit,submission_flair,submission_score,submission_upvote_ratio,submission_date,comment_author,comment_id,comment_score,comment_body,comment_date
0,https://homebrewery.naturalcrit.com/share/TJwo...,IM_Brock,ollo1d,IM_Brock's Barkblood | An ancient migratory pl...,UnearthedArcana,Race,23,1.00,1626456649,IM_Brock,h5f5ctx,2,You can find a link to the Barkblood race [her...,1626456687
1,https://homebrewery.naturalcrit.com/share/SJmS...,cyberhawk94,flm0f1,Sword Mage - 4e's teleporting arcane defender ...,UnearthedArcana,Subclass,1102,0.98,1584667242,cyberhawk94,fkzbprj,20,[**Homebrewery**] (https://homebrewery.natural...,1584667292
2,https://homebrewery.naturalcrit.com/share/Bkri...,cyberhawk94,flm0f1,Sword Mage - 4e's teleporting arcane defender ...,UnearthedArcana,Subclass,1102,0.98,1584667242,cyberhawk94,fkzbprj,20,[**Homebrewery**] (https://homebrewery.natural...,1584667292
3,https://homebrewery.naturalcrit.com/share/ryHL...,cyberhawk94,flm0f1,Sword Mage - 4e's teleporting arcane defender ...,UnearthedArcana,Subclass,1102,0.98,1584667242,cyberhawk94,fkzbprj,20,[**Homebrewery**] (https://homebrewery.natural...,1584667292
4,https://homebrewery.naturalcrit.com/print/ByZd...,cyberhawk94,flm0f1,Sword Mage - 4e's teleporting arcane defender ...,UnearthedArcana,Subclass,1102,0.98,1584667242,cyberhawk94,fkzbprj,20,[**Homebrewery**] (https://homebrewery.natural...,1584667292


# Exploring random comments...

Let's print out some (7) random comments to see how they look...
 
Note several issues apparent here. I'll work through them in order in the following cells.

In [5]:
cmt_sample = cmts_df.sample(n=7, random_state=7)
content = zip(cmt_sample["submission_title"], cmt_sample["comment_body"])

for i, (title, body) in enumerate(content, start=1):
    print(f"\033[91mEntry {i}: " + title + "\033[0m")
    print(f"{body}\n")

Entry 1: The Mystic | A Wisdom Half-Caster that utilises Ki Points
Thanks for your insight :)   
I gave the Mystic the same weapon proficiencies that Wizards and Sorcerers have. I definitely get where you're coming from and it's an idea i did toy with, but I figure most civilians in a fantasy setting are capable of wielding a knife (dagger) or a broom (quarterstaff).  


I love that you're already thinking up new subclasses for the Mystic! I don't necessarily see Jedi Knights being Mystics, mostly because they're not tied to a particular plane of existence, though you could definitely build one using the Aspect of the Traveller and Blessing of the Warrior pathways. I see them more as Monks personally. I've actually designed a Monk subclass based on the Jedi here:  
[https://homebrewery.naturalcrit.com/share/DA31oYNKZ9lD](https://homebrewery.naturalcrit.com/share/DA31oYNKZ9lD)

Entry 2: Revised Monk v1.0. A revision to try to fix the class core problems, including small and big changes 

### Fixing differences in URL formatting

At times URLs are represented diferently (http vs https).

To help make sure all URLs are formatted identically, I wrote a simple function.

`conform_url()` formats all URLs to https://<no www.>domain.extension/path

In [6]:
from src import conform_url

cmts_df["link"] = cmts_df["link"].apply(lambda x: conform_url(x))
subs_df["link"] = subs_df["link"].apply(lambda x: conform_url(x))

In [7]:
index_key = cmts_df.copy()
mapper = index_key["submission_flair"].to_dict()

# Issue #1: Varying link types

We can see we have some different varieties of GMBinder/Homebrewery links, including those with /share/ and /edit/ directories.

There are also some from unrelated domains (Google Docs, Wikipedia articles, etc.) and some direct links to PDF files.

For now we will be dropping any links that aren't directly to 

We also have two varieties of link text:


### URLs in raw text form
URLs with no special formating around them.
> <pre>Gmbinder Link: https://www.gmbinder.com/share/-MW5xKc1QPj4TXEgBzQm</pre>

### Markdown links
These take the form of a display text portion in brackets immediately followed by a URL in parenthesis.
> <pre>[Display text](url)</pre>

In some cases, the display text is also a url and in at least one case early on the url is incorrect in the display text portion, but correct in the actual url segment of the link code.

> <pre>[https://homebrewery.naturalcrit.com/share/MhTK9\_7zHuB6](https://homebrewery.naturalcrit.com/share/MhTK9_7zHuB6)</pre>


Since the url portion is more likely to be correct, we'll go exclusively with raw links and the url portion of the markdown code.

To do this we can use a set of regular expressions. After a few trial and errors, I arrived at the following:

#### #Homebrewery
> <pre>?:https?:\/\/)?(?:www\.)?homebrewery\.naturalcrit\.com\/[\w\-][\w\-]*\/?[\w\-]*\/?[\w\-]*</pre>

#### GMBinder
> <pre>(?:https?:\/\/)?(?:www\.)?gmbinder\.com\/[\w\-][\w\-]*\/?[\w\-]*\/?[\w\-]*[\s\)]</pre>

### Historical Note
The code block below and regular expression described above document a legacy process for handling links within comments. PRAW-CoDiaLS now generates better expressions to automatically collect them.

```
link_re = r'[\s\(](?:https?:\/\/)?(?:www\.)?homebrewery\.naturalcrit\.com\/[\w\-][\w\-]*\/?[\w\-]*\/?[\w\-]*[\s\\)]|[\s\(](?:https?:\/\/)?(?:www\.)?gmbinder\.com\/[\w\-][\w\-]*\/?[\w\-]*\/?[\w\-]*[\s\)]'
cmts_df['link'] = cmts_df['comment_body'].str.findall(link_re)
```

To keep PRAW-CoDiaLS more general-purpose, I still filter certain links below.

In [8]:
bad_substrings = [
    "pdf",  # Remove links to PDFs (GMBinder)
    "profile",  # Remove links to GMBinder user profiles
    "edit",  # Remove links to the main text editor/content creation tool (Homebrewery)
    "print",  # Remove links to PDFs (Homebrewery)
    "user",  # Remove links to Homebrewery user profiles
    "png",  # Image posts
    "-LKYVliROX20cBbSI0r3",  # GMBinder style example
    "-L0WUSjJyFHG7PDyIFta",  # GMbinder tutorial
    "-Kx5i9fuxhuci6BUrbJq",  # GMBiner Watercolor tutorial
    "-MbAAe7gUkgvdAdE47qp",  # Collection of links to Bamstacks' variants
    "B1Hju_QaTx",  # Homebrewery style guide
    "HyWImuH06x",  # Homebrewery styling guide
]
discarded_subs = None
discarded_cmts = None

for substring in bad_substrings:
    cmts_to_drop = cmts_df["link"].str.contains(substring)
    subs_to_drop = subs_df["link"].str.contains(substring)

    discard_dfs = (cmts_df[cmts_to_drop].copy(), subs_df[subs_to_drop].copy())
    discard_dfs[0]["discard_reason"] = "Invalid/Missing Content Type"
    discard_dfs[1]["discard_reason"] = "Invalid/Missing Content Type"

    if (len(discard_dfs[0]) > 0) & isinstance(discarded_cmts, pd.DataFrame):
        discarded_cmts = pd.concat([discarded_cmts, discard_dfs[0]])
    else:
        discarded_cmts = discard_dfs[0]

    if isinstance(discarded_subs, pd.DataFrame) and len(discard_dfs[1]) > 0:
        discarded_subs = pd.concat([discarded_subs, discard_dfs[1]])
    else:
        discarded_subs = discard_dfs[1]

    cmts_df = cmts_df[~cmts_to_drop]
    subs_df = subs_df[~subs_to_drop]

print(f"{len(discarded_cmts)} comments with invalid URL types discarded")
print(f"{len(discarded_subs)} submissions with invalid URL types discarded")

7 comments with invalid URL types discarded
0 submissions with invalid URL types discarded


In [9]:
# Remove links w/o paths (i.e. just to homebrewery.naturalcrit.com or gmbinder.com)
def missing_path(url: str) -> bool:
    return urlparse(url).path == ""


missing_paths = cmts_df["link"].apply(lambda x: missing_path(x))

new_discards = cmts_df[missing_paths].copy()
new_discards["discard_reason"] = "Missing path"
print(f"{len(new_discards)} comments discarded for missing URL path")
discarded_cmts = pd.concat([discarded_cmts, new_discards])
cmts_df = cmts_df[~missing_paths]

33 comments discarded for missing URL path


## Removing known problematic comments based on authorship

Some users will share their related but not necessarily identically-labeled content. Thus, we should only be looking at comments where the submission and comment are made by the same author. Otherwise the flair may not necessarily apply. We can manually check these posts later.


In [10]:
# Make sure that the common author is the same as the post author to help ensure that they refer to the same content.
different_author = cmts_df["submission_author"] != cmts_df["comment_author"]
different_author = cmts_df[different_author].copy()
print(
    f"{len(different_author)} comments with a different author from the primary submission dropped"
)
cmts_df = cmts_df[~cmts_df.index.isin(different_author.index)]

346 comments with a different author from the primary submission dropped


In [11]:
author_issues_path = PRAW / "AuthorIssues.csv"

if author_issues_path.is_file():
    author_issues_df = pd.read_csv(author_issues_path, sep=",", index_col="idx")

    # Check if any links are new.
    links_to_add = different_author[
        ~different_author.index.isin(author_issues_df.index)
    ]
    print(
        f"Merged {len(links_to_add)} links into existing set of {len(author_issues_df)}"
    )

    # Add only new links
    author_issues_df = pd.concat([author_issues_df, links_to_add])
    author_issues_df["manually_reviewed"] = author_issues_df[
        "manually_reviewed"
    ].fillna(False)

else:
    author_issues_df = different_author
    author_issues_df["manually_reviewed"] = False

Merged 0 links into existing set of 346


In [12]:
print(
    f"Saving {len(author_issues_df)} comment links where the author differs from the submission for manual review"
)
print(
    f"{author_issues_df['manually_reviewed'].sum()} out of {len(author_issues_df)} comments with authorship issues have been reviewed already."
)
author_issues_df.to_csv(author_issues_path, index_label="idx")

Saving 346 comment links where the author differs from the submission for manual review
346 out of 346 comments with authorship issues have been reviewed already.


# Issue #2: Lack of 1:1 correspondence between links and submissions
For several reasons, a given piece of user content can be associated with multiple submissions/comments and vice versa.
I'll go through them one by one below.

#### General Notes:
* The main idea here is to have a nice variety of documents with meaningful flair, so I'll always preserve UnearthedArcana links when present and never remove an UnearthedArcana link in favor of a DnDHomebrew one.
* I'm going to be keeping track of the rows I drop in case I decide to manually inspect for QC later on.

##### All the functions defined below for filtering out repeat links will <u>**modify** the main dataframe(s) **inplace**</u> and <u>**return** any discarded rows</u>.
Individual functions described and imported below:
* `filter_simple_cmt_issues()`
* `get_multi_link_parents_by_author()`
* `filter_cmt_multi_links_by_date()`

### Issue #2: Part A -- Simple Issues (obvious redundancies)

There are two obvious types of easily resolvable redundancies I can identify.

1. Submissions where a link is posted in the comments multiple times.
2. Links collected from comments, where a meaningfully flaired primary submission was also found

This functionality is implemented in the `filter_simple_cmt_issues()` function


In [13]:
from src import filter_simple_cmt_issues

new_discards = filter_simple_cmt_issues(cmts_df, subs_df)
discarded_cmts = pd.concat([discarded_cmts, new_discards])

Removed 238 duplicated linkes from comments with the same fromparent submission ID and link
Removed 215 links from comments that have primary submissions on /r/UnearthedArcana


### Issue #2: Part B -- Cross-posts
Some content has been shared on both /r/DnDHomebrew and /r/UnearthedArcana.

Since /r/UnearthedArcana has useful flair and /r/DnDHomebrew does not, we will always prioritize /r/UnearthedArcana links.

A helper function for this, `filter_dh_cross_posts()`, has been defined.

In [14]:
# Import function to remove links posted to /r/DnDHomebrew if they are already available from /r/UnearthedArcana
from src import filter_dh_cross_posts

new_discards = filter_dh_cross_posts(cmts_df, subs_df)
discarded_cmts = pd.concat([discarded_cmts, new_discards[0]])
discarded_subs = pd.concat([discarded_subs, new_discards[1]])

Removed 302 /r/DnDHomebrew comments that were also cross-posted to /r/UnearthedArcana
Removed 56 /r/DnDHomebrew submissions that were also cross-posted to /r/UnearthedArcana


### Issue #2: Part C -- Multi-link comments
Frequently an author will self-promote their other content within a comment. We'll need to identify and resolve these situations somehow later on.

For instance take the comment with ID *fkzbprj* under the submission titled "Sword Mage - 4e's teleporting arcane defender brought back as an Artificer subclass - v2.1" where the author, *cyberhawk92*, links to a grand total of **14** other piees of content they've created.

I've defined two functions to help with the process of cleaning up some of these cases.

`get_multi_link_parents_by_author()` creates a dictionary with authors as keys and lists of all posts by that author as values. Only authors with >1 post are included.

`filter_cmt_multi_links_by_date()` goes through and discards comments where the link has been repeated but was already posted in an older comment (more likely to be the original)

In [15]:
from src import filter_cmt_multi_links_by_date

newer_multi_cmts = filter_cmt_multi_links_by_date(cmts_df)
discarded_cmts = pd.concat([discarded_cmts, newer_multi_cmts])
cmts_df = cmts_df[~cmts_df.index.isin(newer_multi_cmts.index)]
newer_multi_cmts.sort_values(by="link")

Found 1087 links with shared parent submissions (across 195 total submissions)
Discarded 350 links from authors with multiple link comments in favor of an older post associated with the same link.


,link,submission_author,submission_id,submission_title,subreddit,submission_flair,submission_score,submission_upvote_ratio,submission_date,comment_author,comment_id,comment_score,comment_body,comment_date,discard_reason
3157,https://gmbinder.com/share/-Lg5FRCxWb9eArTQf9R3,Thudnfer,wddi96,"Path Of Embers - Ashes To Ashes, And Coal...To...",UnearthedArcana,Subclass,56,0.96,1659347829.0,Thudnfer,iihmvc3,1,"OH GOD THE FIRE BURNS, WHY DIDNT ANYONE TELL M...",1659348006.0,Newer multi-comment
272,https://gmbinder.com/share/-Ln4qu88tNDwBKVrxrAt,KylenBlaise,lma184,[5e] The Death Knight V5.5: Come back to life ...,UnearthedArcana,Class,590,0.98,1613610901,KylenBlaise,gnu0u5s,13,Greetings! For those who have not seen my work...,1613610927,Newer multi-comment
2241,https://gmbinder.com/share/-Lt8HrxbsTha-N2o8bzz,Thudnfer,skot7d,"Lekras' Embrace - Oh Boy, I Hope Nobody Sucks ...",UnearthedArcana,Spell,43,0.89,1644009960,Thudnfer,hvm6ejs,2,This is literally just text so no PDF link is ...,1644010053,Newer multi-comment
3243,https://gmbinder.com/share/-M0ZVK6ndhFyImQPF_aJ,LaserLlama,zk7814,laserllama's Alternate Rogue (Update) - Become...,UnearthedArcana,Class,69,0.97,1670870068.0,LaserLlama,izyhjt4,1,"If you like the Holmes archetype, check out th...",1670875089.0,Newer multi-comment
2380,https://gmbinder.com/share/-M0dBqq372X_oBPUT4vP,LaserLlama,wctd3z,Alternate Barbarian: Expanded - A Multitude of...,UnearthedArcana,Class,441,0.98,1659286744.0,LaserLlama,iiegz0m,7,**This is an expansion for my [Alternate Barba...,1659286797.0,Newer multi-comment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3420,https://homebrewery.naturalcrit.com/share/zSvN...,vonBoomslang,yzq1uk,Improved Invisibility - Not-As-Great-As-Greate...,UnearthedArcana,Spell,754,0.97,1668902762.0,vonBoomslang,ix1dh2u,13,"Yet another of my ""haha this is a silly spell,...",1668902874.0,Newer multi-comment
3561,https://homebrewery.naturalcrit.com/share/zSvN...,vonBoomslang,ywe4yv,"Ball Lightning - Powerful, but unpredictable.",UnearthedArcana,Spell,309,0.99,1668557562.0,vonBoomslang,iwj25fq,17,"It's funny, how an experience can be formative...",1668557569.0,Newer multi-comment
3933,https://homebrewery.naturalcrit.com/share/zSvN...,vonBoomslang,yo8qo8,"Fireball - Still too strong, but now approachi...",UnearthedArcana,Spell,4,0.57,1667783961.0,vonBoomslang,ivcyxuo,3,"I got asked to share my fireball rework (nerf,...",1667784157.0,Newer multi-comment
3595,https://homebrewery.naturalcrit.com/share/zSvN...,vonBoomslang,z7ejgc,"Summon Crystalkin - So pretty, and glittery, a...",UnearthedArcana,Spell,162,0.99,1669683248.0,vonBoomslang,iy64qvq,3,I'd been wanting to make a Tasha-style summon ...,1669683440.0,Newer multi-comment


### Issue #2: Part D -- Exporting the remaining multi-link comments
I'm going to export a csv of the comments w/ multiple links for a manual review.

In [16]:
from src import get_multi_link_parents_by_author

multi_link_dict = get_multi_link_parents_by_author(cmts_df)
offenders = [v for v in multi_link_dict.values()]
print(f"{len(offenders)} remaining authors with repeat posts")

Found 725 links with shared parent submissions (across 147 total submissions)
92 remaining authors with repeat posts


In [17]:
offenders = list(itertools.chain(*offenders))
multi_link_df = cmts_df[cmts_df["submission_id"].isin(offenders)]
cmts_df = cmts_df[~cmts_df["submission_id"].isin(offenders)]
multi_link_df

,link,submission_author,submission_id,submission_title,subreddit,submission_flair,submission_score,submission_upvote_ratio,submission_date,comment_author,comment_id,comment_score,comment_body,comment_date
3342,https://homebrewery.naturalcrit.com/share/1Pe-...,Oh_Hi_Mark_,ziz7wl,Give me a D&D monster and I'll make you a bett...,DnDHomebrew,5e,31,0.81,1670777336.0,Oh_Hi_Mark_,izxvbcc,2,* [Grungs](https://homebrewery.naturalcrit.com...,1670866663.0
3325,https://homebrewery.naturalcrit.com/share/1x9v...,Oh_Hi_Mark_,ziz7wl,Give me a D&D monster and I'll make you a bett...,DnDHomebrew,5e,31,0.81,1670777336.0,Oh_Hi_Mark_,izw8r08,4,"All right, here's all the DiA stuff I can fit ...",1670833816.0
3332,https://homebrewery.naturalcrit.com/share/149M...,Oh_Hi_Mark_,ziz7wl,Give me a D&D monster and I'll make you a bett...,DnDHomebrew,5e,31,0.81,1670777336.0,Oh_Hi_Mark_,izw8x45,1,* [Rust Monsters](https://homebrewery.naturalc...,1670833960.0
3329,https://homebrewery.naturalcrit.com/share/1_Ll...,Oh_Hi_Mark_,ziz7wl,Give me a D&D monster and I'll make you a bett...,DnDHomebrew,5e,31,0.81,1670777336.0,Oh_Hi_Mark_,izw8r08,4,"All right, here's all the DiA stuff I can fit ...",1670833816.0
3328,https://homebrewery.naturalcrit.com/share/11oG...,Oh_Hi_Mark_,ziz7wl,Give me a D&D monster and I'll make you a bett...,DnDHomebrew,5e,31,0.81,1670777336.0,Oh_Hi_Mark_,izw8r08,4,"All right, here's all the DiA stuff I can fit ...",1670833816.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,https://homebrewery.naturalcrit.com/share/ryWT...,SethBlackwood,be9i3w,Oath of the Grim Hunt // A paladin oath devote...,UnearthedArcana,Subclass,1432,0.98,1555516791,SethBlackwood,el40hdx,98,~~Edit: It is utterly embarrassing that I didn...,1555517006
299,https://gmbinder.com/share/-LZWXq83Rk2jZNs7spok,aeyana,b5c7px,[Monster] The Book of Hordes - Rules for Mass ...,UnearthedArcana,Monster,1366,0.99,1553528483,aeyana,ejcfn03,90,*The Book of Hordes* is a monster and military...,1553528601
300,https://gmbinder.com/share/-LarKZIy-8s4mQWAwy4x,aeyana,b5c7px,[Monster] The Book of Hordes - Rules for Mass ...,UnearthedArcana,Monster,1362,0.99,1553528483,aeyana,ejdqh18,40,It's not a big ask at all! I just [got one set...,1553559675
82,https://homebrewery.naturalcrit.com/share/SJGH...,xpertranger,ark640,The way of the Frozen Fist- An ice-themed mona...,UnearthedArcana,Subclass,880,0.98,1550406797,xpertranger,ego41ix,5,Wow this got a bunch of attention. Gonna hijac...,1550422117


In [18]:
multi_link_path = PRAW / "MultiLinkCmts.csv"

if multi_link_path.is_file():
    _multi_link_df = pd.read_csv(multi_link_path, index_col="idx")
    print(_multi_link_df.index.isin(cmts_df.index).sum() / len(_multi_link_df))
    missing_cmts = multi_link_df[~multi_link_df.index.isin(_multi_link_df.index)]
    print(
        f"Merged {len(missing_cmts)} links into existing set of {len(_multi_link_df)}"
    )
    multi_link_df = pd.concat([_multi_link_df, missing_cmts])
    multi_link_df["manually_reviewed"] = multi_link_df["manually_reviewed"].fillna(
        False
    )
else:
    multi_link_df["manually_reviewed"] = False
multi_link_df = multi_link_df.sort_index()

0.041666666666666664
Merged 0 links into existing set of 936


In [19]:
print(
    f"Saving {len(multi_link_df)} links from comments with mutliple links for manual review."
)
print(
    f"{multi_link_df['manually_reviewed'].sum()} out of {len(multi_link_df)} links from comments with multiple associated links have been reviewed already."
)
multi_link_df.to_csv(multi_link_path, index_label="idx")

Saving 936 links from comments with mutliple links for manual review.
936 out of 936 links from comments with multiple associated links have been reviewed already.


### Historical Note
Some links were previously manually reviewed for being part of "multi link comments" but changes in the pipeline and/or collection of additional led to the other links associated with these comments being filtered out earlier or otherwise changing their status.

Nonetheless, I'd rather go with the manually reviewed labels since comments can still be mis-associated with submission flair and this adds an additional layer of certainty for even more labels.

In [20]:
print(
    f"Dropped {cmts_df.index.isin(multi_link_df.index).sum()} additional comments already processed in mutli-link comment file"
)
cmts_df = cmts_df[~cmts_df.index.isin(multi_link_df.index)]

Dropped 39 additional comments already processed in mutli-link comment file


Un-discard any manually reviewed content (i.e. in previous iterations of dataset present in the multi-link file).

Check why they were discarded.

In [21]:
discard_reasons = discarded_cmts[discarded_cmts.index.isin(multi_link_df.index)][
    "discard_reason"
].value_counts()
discarded_cmts = discarded_cmts[~discarded_cmts.index.isin(multi_link_df.index)]
discard_reasons

Newer multi-comment    169
Simple duplicate         3
Name: discard_reason, dtype: int64

In [22]:
discarded_cmts.to_csv(PRAW / "CommentsDropped.csv", index_label="idx")
cmts_df.to_csv(PRAW / "CommentsFiltered.csv", index_label="idx")

### Filtering redundant primary submissions
Instances where a link has been submitted more than once. In these cases use the most recent /r/UnearthedArcana version since it's most likely to be correct.

In [23]:
subs_to_drop = subs_df.sort_values(
    by=["link", "subreddit", "submission_date"], axis=0, ascending=[True, False, False]
)
subs_to_drop = subs_to_drop[
    subs_to_drop.duplicated(subset=["link", "submission_flair"], keep="first")
]
subs_to_drop["discard_reason"] = "Repeated link with newer submission"
discarded_subs = pd.concat([discarded_subs, subs_to_drop])

In [24]:
subs_df = subs_df[~subs_df.index.isin(subs_to_drop.index)]
remaining_dups = subs_df.duplicated(subset="link", keep=False).sum()
print(f"Dropped {len(subs_to_drop)} duplicated links submitted as primary submissions")
print(f"{remaining_dups} duplicate submissions remaining")

Dropped 25 duplicated links submitted as primary submissions
2 duplicate submissions remaining


Let's inspect the remaining duplicate submission links

In [25]:
subs_df[subs_df.duplicated(subset="link", keep=False)]

,link,submission_author,submission_id,submission_title,subreddit,submission_flair,submission_score,submission_upvote_ratio,submission_date,comment_author,comment_id,comment_score,comment_body,comment_date
658,https://gmbinder.com/share/-MG_WJhrrG6ykwrgs_tO,Dracovitch,srrb9f,Shadowheart Apocrypha: Classes (v2.0)! 4 brand...,UnearthedArcana,Compendium,21.0,1.0,1.644782e+09,NaN,NaN,NaN,NaN,NaN
823,https://gmbinder.com/share/-MG_WJhrrG6ykwrgs_tO,Dracovitch,z9yv7c,Shadowheart Apocrypha: Classes (v9.0)! 5 brand...,UnearthedArcana,Class,67.0,1.0,1.669927e+09,NaN,NaN,NaN,NaN,NaN


In [26]:
subs_df[subs_df.duplicated(subset="link", keep=False)]["link"].values[0]

'https://gmbinder.com/share/-MG_WJhrrG6ykwrgs_tO'

Both of these appear to be somewhat accurate. It's a compendium of classes (which has grown since the initial post). I'll keep the compendium link.

In [27]:
subs_df = subs_df[subs_df.index != 823]

In [28]:
discarded_subs.to_csv(PRAW / "SubmissionsDropped.csv", index_label="idx")
subs_df.to_csv(PRAW / "SubmissionsFiltered.csv", index_label="idx")